<a href="https://colab.research.google.com/github/bootcampshare/pickles/blob/master/JD_NC_Price_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%%capture
#!pip install alpaca-py
import pip
def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])

import_or_install('alpaca-py')

api_key = 'AKP5MUNUSKFEV2B5MPC1'
api_secret_key='JhbmmBlj506tdEFZ5PzurwrJmiis9oHLg0D6U11X'

In [24]:
## this method uses the MARKET DATA api

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.requests import StockBarsRequest
import pandas as pd

data_client = StockHistoricalDataClient(api_key, api_secret_key)

# Set parameters
symbols=['QQQ']
minute_bars =  pd.DataFrame()
for year in ['2023']: #['2016','2017','2018','2019','2020','2021','2022','2023']:
  start_date = pd.to_datetime(year+"-01-01").tz_localize('America/Chicago')
  end_date = pd.to_datetime(year+"-12-31").tz_localize('America/Chicago')
  request_parameters = StockBarsRequest(symbol_or_symbols=symbols, timeframe=TimeFrame(1, TimeFrameUnit.Minute), start=start_date, end=end_date,)
  # Fetch data and convert to dataframe
  if minute_bars.empty:
    minute_bars = data_client.get_stock_bars(request_parameters).df
  else: minute_bars = pd.concat([minute_bars, data_client.get_stock_bars(request_parameters).df])

In [21]:
## EXPORT SECTION
## export to csv for github
bars = minute_bars.reset_index()
#bars['timestamp'] = bars.timestamp.dt.strftime('%Y-%m-%dT%H:%M:%S')
#bars[['timestamp', 'volume', 'trade_count', 'vwap']].to_csv('/content/QQQ.csv')

In [11]:
bars

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,QQQ,2016-01-04T09:00:00,110.24,110.24,110.21,110.22,1686.0,22.0,110.231696
1,QQQ,2016-01-04T09:02:00,110.24,110.24,110.24,110.24,112.0,1.0,110.240000
2,QQQ,2016-01-04T09:03:00,110.16,110.16,110.15,110.15,1320.0,3.0,110.156689
3,QQQ,2016-01-04T09:05:00,110.08,110.08,110.08,110.08,202.0,1.0,110.080000
4,QQQ,2016-01-04T09:09:00,110.00,110.00,109.99,109.99,220.0,3.0,109.995455
...,...,...,...,...,...,...,...,...,...
1470401,QQQ,2023-12-30T00:53:00,409.06,409.06,409.06,409.06,325.0,9.0,409.046861
1470402,QQQ,2023-12-30T00:54:00,409.02,409.02,409.02,409.02,203.0,15.0,409.024926
1470403,QQQ,2023-12-30T00:57:00,409.01,409.03,409.00,409.00,726.0,24.0,409.011371
1470404,QQQ,2023-12-30T00:58:00,409.00,409.02,409.00,409.02,767.0,11.0,409.012777


In [160]:

bars = minute_bars.reset_index()
bars['date'] = pd.to_datetime(bars["timestamp"]).dt.date

bars = bars.assign(cumAvgvwap=bars.groupby('date', sort=False)['vwap'].transform(lambda x: x.expanding().mean()))
bars['cumVolume'] = bars.groupby('date', sort=False)['volume'].cumsum()
bars['totalPrice'] = bars.volume * bars.vwap
bars['cumPrice'] = bars.groupby('date', sort=False)['totalPrice'].cumsum()
bars['dvwap'] = bars.cumPrice / bars.cumVolume
bars

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap,date,cumAvgvwap,cumVolume,totalPrice,cumPrice,dvwap
0,QQQ,2024-01-05 09:00:00+00:00,395.5100,395.81,395.45,395.45,839.0,36.0,395.606806,2024-01-05,395.606806,839.0,3.319141e+05,3.319141e+05,395.606806
1,QQQ,2024-01-05 09:01:00+00:00,395.5400,395.54,395.54,395.54,535.0,15.0,395.544467,2024-01-05,395.575636,1374.0,2.116163e+05,5.435304e+05,395.582533
2,QQQ,2024-01-05 09:02:00+00:00,395.5100,395.61,395.51,395.61,1200.0,17.0,395.547558,2024-01-05,395.566277,2574.0,4.746571e+05,1.018187e+06,395.566228
3,QQQ,2024-01-05 09:03:00+00:00,395.5000,395.50,395.50,395.50,1002.0,4.0,395.500329,2024-01-05,395.549790,3576.0,3.962913e+05,1.414479e+06,395.547763
4,QQQ,2024-01-05 09:04:00+00:00,395.4300,395.43,395.30,395.30,3663.0,23.0,395.357931,2024-01-05,395.511418,7239.0,1.448196e+06,2.862675e+06,395.451706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,QQQ,2024-01-06 00:55:00+00:00,396.8700,396.87,396.87,396.87,444.0,7.0,396.868287,2024-01-06,396.855499,15461.0,1.762095e+05,6.135773e+06,396.854878
774,QQQ,2024-01-06 00:56:00+00:00,396.9000,396.90,396.90,396.90,980.0,11.0,396.897423,2024-01-06,396.858294,16441.0,3.889595e+05,6.524733e+06,396.857414
775,QQQ,2024-01-06 00:57:00+00:00,396.8800,396.90,396.88,396.90,3452.0,18.0,396.898895,2024-01-06,396.860832,19893.0,1.370095e+06,7.894828e+06,396.864612
776,QQQ,2024-01-06 00:58:00+00:00,396.9000,396.95,396.90,396.95,1213.0,15.0,396.941181,2024-01-06,396.865558,21106.0,4.814897e+05,8.376317e+06,396.869013


In [ ]:
# OHLC prices by 133 ticks
#df = pd.read_csv('https://github.com/bootcampshare/pickles/blob/master/appl5.csv', index_col=0)
#bd = df.copy()
#bd = pd.read_csv('basic_data_052420.csv', index_col=0)